In [ ]:
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import sys
sys.path.append('..')

In [ ]:
import fetch_missing_chirps_files as fmcf
import chcfetch.constants
import read_tifs_create_met as rtcm

In [ ]:
YEARS = [1981, 1982]

In [ ]:
GEOGLAM_CHIRPS_FOLDERPATH = '../../../../nh_crop_calendar/crop_calendar/data/cluster_files/chirps/'
CHC_CHIRPS_DOWNLOAD_FOLDERPATH = '../../../../nh_crop_calendar/crop_calendar/data/chc/chirps-v2.0/p05/'

In [ ]:
CHIRPS_REF_TIF_FILEPATH = '../../../../nh_crop_calendar/crop_calendar/data/cluster_files/chirps/chirps_2019/global/chirps_v2.0.2019001_global.tif'

In [ ]:
os.path.exists(CHIRPS_REF_TIF_FILEPATH)

In [ ]:
chirps_bounds_gdf = gpd.read_file(chcfetch.constants.CHIRPS_V2_P50_BOUNDS_GEOJSON_FILEPATH)

In [ ]:
chirps_bounds_gdf

In [ ]:
eth_gdf = gpd.read_file('../../../../ethiopia/data/Eth_Woreda_2013_UTM/Eth_Woreda_2013_UTM.shp')

In [ ]:
eth_gdf.to_crs('epsg:4326').iloc[[eth_gdf.area.argmin()]]

In [ ]:
smallest_zone_gdf = eth_gdf.to_crs('epsg:4326').iloc[[eth_gdf.area.argmin()],].reset_index(drop=True)

In [ ]:
smallest_zone_gdf['geometry'][0]

In [ ]:
smallest_zone_gdf.to_file('../data/eth_smalled_shape.geojson')

In [ ]:
smallest_zone_gdf.within(chirps_bounds_gdf['geometry'][0])

In [ ]:
catalogue_df = fmcf.fetch_missing_chirps_v2p0_p05_files(
    years = YEARS,
    geoglam_chirps_data_folderpath = GEOGLAM_CHIRPS_FOLDERPATH,
    chc_chirps_v2_0_p05_download_folderpath = CHC_CHIRPS_DOWNLOAD_FOLDERPATH,
    # njobs = njobs,
    # overwrite = overwrite,
    # tif_filepath_col = tif_filepath_col,
)

In [ ]:
catalogue_df['filetype'].value_counts()

In [ ]:
catalogue_df

In [ ]:
catalogue_df[rtcm.METHOD_COL] = rtcm.LoadTIFMethod.READ_AND_CROP
catalogue_df.loc[
    catalogue_df[fmcf.FILETYPE_COL] == fmcf.TIF_GZ_EXT,
    rtcm.METHOD_COL
] = rtcm.LoadTIFMethod.COREGISTER_AND_CROP

In [ ]:
catalogue_df['method'].value_counts()

In [ ]:
# njobs = 8 -> 39.2s
# njobs = 1 -> 3m 15.1s

updated_catalogue_df = rtcm.read_tifs_get_agg_value(
    shapes_gdf = smallest_zone_gdf,
    catalogue_df = catalogue_df,
    val_col = 'mean chirps',
    reference_tif_filepath = CHIRPS_REF_TIF_FILEPATH,
    # njobs = 1,
    working_folderpath = '../data/temp_working_dir'
)

In [ ]:
scale = 5
aspect_ratio = 3
fig, ax = plt.subplots(figsize=(scale*aspect_ratio, scale))
sns.lineplot(
    ax = ax,
    data = updated_catalogue_df,
    x = 'date',
    y = 'mean chirps',
)

In [ ]:
updated_catalogue_df

In [ ]:
list(range(2020, 2020+1))

In [ ]:
"""
- roi_shapefile
- start_year
- end_year
- geoglam_chirps_folderpath
- chc_chirps_download_folderpath
- reference_tif_filepath
- aggregation
- njobs
- export_filepath
"""

(
    "./data/eth_smalled_shape.geojson "
    "2019 "
    "2023 "
    "../../../nh_crop_calendar/crop_calendar/data/cluster_files/chirps/ "
    "../../../nh_crop_calendar/crop_calendar/data/chc/chirps-v2.0/p05 "
    "../../../nh_crop_calendar/crop_calendar/data/cluster_files/chirps/... "
    "mean "
    "-1 "
    "./data/test_out.csv"
    "./data/temp_working_dir/"
)

In [ ]:
import pandas as pd

In [ ]:
out_df = pd.read_csv('../data/Addis Ketema CHIRPS.csv')

In [ ]:
out_df

In [ ]:
import datetime

In [ ]:
out_df['date'] = out_df['date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))

In [ ]:
scale = 5
aspect_ratio = 4
fig, ax = plt.subplots(figsize=(scale*aspect_ratio, scale))
sns.lineplot(
    ax = ax,
    data = out_df,
    x = 'date',
    y = 'mean CHIRPS',
)
ax.set_title('Addis Ketema, Ethiopia, mean CHIRPS, 1981 - present')